In [ ]:
import pandas as pd
import numpy as np

path_to_data_folder = "../Data/"

# خواندن داده‌ها
train = pd.read_csv(path_to_data_folder + "train.csv")
test = pd.read_csv(path_to_data_folder + "test.csv")

In [ ]:
for col in train.columns:
    print(f"-- {col} -- ")
    print(train[col].info(), end="\n\n")
    print(train[col].describe(), end="\n\n")
    print(train[col].head(3))
    print("--------------------", end="\n\n")

-- userID -- 
<class 'pandas.core.series.Series'>
RangeIndex: 15235 entries, 0 to 15234
Series name: userID
Non-Null Count  Dtype 
--------------  ----- 
15235 non-null  object
dtypes: object(1)
memory usage: 119.2+ KB
None

count     15235
unique     1000
top       V0799
freq         27
Name: userID, dtype: object

0    V0921
1    V0286
2    V0004
Name: userID, dtype: object
--------------------

-- restaurantID -- 
<class 'pandas.core.series.Series'>
RangeIndex: 15235 entries, 0 to 15234
Series name: restaurantID
Non-Null Count  Dtype 
--------------  ----- 
15235 non-null  object
dtypes: object(1)
memory usage: 119.2+ KB
None

count     15235
unique       50
top        R034
freq        336
Name: restaurantID, dtype: object

0    R015
1    R050
2    R024
Name: restaurantID, dtype: object
--------------------

-- RName -- 
<class 'pandas.core.series.Series'>
RangeIndex: 15235 entries, 0 to 15234
Series name: RName
Non-Null Count  Dtype 
--------------  ----- 
15235 non-null  object
dt

In [ ]:
cols_to_drop = ["userID", "restaurantID", "RName"]
train = train.drop(columns=cols_to_drop)
test = test.drop(columns=cols_to_drop)

In [ ]:
print(train.isnull().sum())
print(test.isnull().sum())

Cuisine              0
Area                 0
Seats                0
Halls_Count          0
Summer_Terrace       0
VIP_Zone             0
Avg_Bill             0
Staff_Count          0
Michelin_Stars       0
Age                  0
Gender               0
Profession           0
Openness             0
Conscientiousness    0
Extraversion         0
Agreeableness        0
Neuroticism          0
Rating               0
dtype: int64
Cuisine              0
Area                 0
Seats                0
Halls_Count          0
Summer_Terrace       0
VIP_Zone             0
Avg_Bill             0
Staff_Count          0
Michelin_Stars       0
Age                  0
Gender               0
Profession           0
Openness             0
Conscientiousness    0
Extraversion         0
Agreeableness        0
Neuroticism          0
dtype: int64


In [ ]:
# اضافه کردن ستون هدف به نسخه‌ای از train برای حفظ ساختار
train_features = train.drop(columns="Rating")
combined = pd.concat([train_features, test], axis=0)

# ستون‌های دسته‌ای
categorical_cols = ["Cuisine", "Gender", "Profession", "Summer_Terrace", "VIP_Zone"]

# One-Hot Encoding
combined_encoded = pd.get_dummies(combined, columns=categorical_cols, drop_first=True)

# جدا کردن دوباره مجموعه‌ها
X = combined_encoded.iloc[: len(train)]
X_test = combined_encoded.iloc[len(train) :]

# y فقط از train گرفته می‌شود
y = train["Rating"]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# تقسیم آموزش/اعتبارسنجی
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# آموزش مدل
model = LinearRegression()
model.fit(X_train, y_train)

# پیش‌بینی و ارزیابی
y_pred = model.predict(X_val)

r2_train = r2_score(y_train, model.predict(X_train))
r2_val = r2_score(y_val, y_pred)

rmse_train = np.sqrt(mean_squared_error(y_train, model.predict(X_train)))
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred))

print(f"R2 (Train): {r2_train:.4f}")
print(f"R2 (Validation): {r2_val:.4f}")
print(f"RMSE (Train): {rmse_train:.4f}")
print(f"RMSE (Validation): {rmse_val:.4f}")

R2 (Train): 0.8830
R2 (Validation): 0.8813
RMSE (Train): 0.3393
RMSE (Validation): 0.3415


In [ ]:
# پیش‌بینی امتیاز روی داده تست
test_predictions = model.predict(X_test)

# ساخت DataFrame خروجی
submission = pd.DataFrame({"Rating": test_predictions})